In [ ]:
#Download and Import Dependencies
#!pip install tensorflow
import tensorflow as tf
print(tf.__version__)
# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
# example of loading the vgg16 model
#from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
# load model
#model = MobileNetV2()
# summarize the model
#model.summary()

In [ ]:
# Download the Fashion Dataset 
fashion_mnist = tf.keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

train_images = train_images.reshape((train_images.shape[0], 28, 28, 1))
test_images = test_images.reshape((test_images.shape[0], 28, 28, 1))



class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

In [ ]:
#Explore the Data

print('TRAIN DATA FORMAT (samples, width, height): {}'.format(train_images.shape))
print('TEST DATA FORMAT (samples, width, height): {}'.format(test_images.shape))

In [ ]:
#Visualize a sample image

plt.figure()
plt.imshow(train_images[0][:,:,0])
plt.colorbar()
plt.grid(False)
plt.show()

In [ ]:
#Scale image pixel values from [0-255] to [0-1]

train_images = train_images / 255.0
test_images = test_images / 255.0

In [ ]:
plt.figure(figsize=(20,20))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i][:,:,0], cmap=plt.cm.binary)
    plt.xlabel(class_names[train_labels[i]])
plt.show()


In [ ]:
# Build a Convolutional Neural Network (CNN) Model

model = tf.keras.Sequential([
    
    tf.keras.layers.Conv2D(28, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    
    #tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    #tf.keras.layers.MaxPooling2D((2, 2)),
    
    tf.keras.layers.Flatten(),
    
    #tf.keras.layers.Dense(256, activation='relu'),
    #tf.keras.layers.Dropout(rate=0.2),
    
    #tf.keras.layers.Dense(128, activation='relu'),
    #tf.keras.layers.Dropout(rate=0.2),
    
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10)
])
    
    

model.summary()

In [ ]:
#Visualize the CNN layers

dot_img_file = 'model_1.png'
tf.keras.utils.plot_model(model, to_file=dot_img_file, show_shapes=True)

In [ ]:
#Compile the CNN model

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


In [ ]:
#Train the CNN model

history=model.fit(train_images,
                  train_labels, 
                  batch_size=256, 
                  epochs=5, 
                  verbose=1, 
                  validation_split=0.2,
                  shuffle=True)
                 

In [ ]:
#Plot changes in Training and Validation losses

plt.plot(history.history['loss'], label='train_loss')
plt.plot(history.history['val_loss'], label = 'val_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.ylim([0., 1])
plt.legend(loc='lower right')

In [ ]:
#Evaluate the Test Loss

test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

In [ ]:
#Util Functions for some drawings

def plot_image(i, predictions_array, true_label, img):
  true_label, img = true_label[i], img[i][:,:,0]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])

  plt.imshow(img, cmap=plt.cm.binary)

  predicted_label = np.argmax(predictions_array)
  if predicted_label == true_label:
    color = 'blue'
  else:
    color = 'red'

  plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                class_names[true_label]),
                                color=color)

def plot_value_array(i, predictions_array, true_label):
  true_label = true_label[i]
  plt.grid(False)
  plt.xticks(range(10))
  plt.yticks([])
  thisplot = plt.bar(range(10), predictions_array, color="#777777")
  plt.ylim([0, 1])
  predicted_label = np.argmax(predictions_array)

  thisplot[predicted_label].set_color('red')
  thisplot[true_label].set_color('green')


In [ ]:
#Get the Predictions with Probabilities

probability_model = tf.keras.Sequential([model, tf.keras.layers.Softmax()])
predictions = probability_model.predict(test_images)


# Plot the first X test images, their predicted labels, and the true labels.
# Color correct predictions in blue and incorrect predictions in red.
num_rows = 10
num_cols = 2
num_images = num_rows*num_cols
plt.figure(figsize=(2*4*num_cols, 4*num_rows))
for i in range(num_images):
  plt.subplot(num_rows, 2*num_cols, 2*i+1)
  plot_image(i, predictions[i], test_labels, test_images)
  plt.subplot(num_rows, 2*num_cols, 2*i+2)
  plot_value_array(i, predictions[i], test_labels)
  _ = plt.xticks(range(10), class_names, rotation=45)

plt.tight_layout()
plt.show()